### Install and Import Dependencies

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ---------------------------------------- 0.0/2.4 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.4 GB 2.6 MB/s eta 0:15:14
     ---------------------------------------- 0.0/2.4 GB 3.2 MB/s eta 0:12:46
     ---------------------------------------- 0.0/2.4 GB 4.3 MB/s eta 0:09:27
     ---------------------------------------- 0.0/2.4 GB 6.8 MB/s eta 0:05:54
     ---------------------------------------- 0.0/2.4 GB 11.0 MB/s eta 0:03:39
     ---------------------------------------- 0.0/2.4 GB 12.9 MB/s eta 0:03:07
     ---------------------------------------- 0.0/2.4 GB 14.6 MB/s eta 0:02:46
     ---------------------------------------- 0.0/2.4 GB 15.4 MB/s eta 0:02:37
     ---------------------------------------- 0.0/2.4 GB 13.6 MB/s eta 0:02:57
     ---------------------------------------- 0.0/2.4 GB 17.0 MB/s eta 0:02:22
     ---------------------------------------- 0.0/2.4 GB 18.2 MB/s eta 0:02:12
     


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install transformers

     ---------------------------------------- 0.0/138.0 kB ? eta -:--:--
     -------------------------------------- 138.0/138.0 kB 8.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB ? eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB ? eta 0:00:00
   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   ---- ----------------------------------- 0.9/9.0 MB 30.1 MB/s eta 0:00:01
   ------------ --------------------------- 2.8/9.0 MB 35.2 MB/s eta 0:00:01
   ------------- -------------------------- 3.0/9.0 MB 24.0 MB/s eta 0:00:01
   ----------------- ---------------------- 3.9/9.0 MB 22.8 MB/s eta 0:00:01
   --------------------- ------------------ 4.8/9.0 MB 21.9 MB/s eta 0:00:01
   -------------------------- ------------- 5.9/9.0 MB 22.3 MB/s eta 0:00:01
   -----------------------


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
import pandas as pd
import numpy as np

d:\Study\ORIE_24SP\PAML\streamlitenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Instantiate Model

In [4]:
# 加载预训练的 tokenizer
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 加载预训练的模型
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

d:\Study\ORIE_24SP\PAML\streamlitenv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\Study\ORIE_24SP\PAML\streamlitenv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Owner\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer 

### Encode and Calculate Sentiment

In [5]:
# 文本编码
tokens = tokenizer.encode('I hated this, absolutely the worst', return_tensors='pt')

In [21]:
# 预测结果
result = model(tokens)
result


# SequenceClassifierOutput(loss=None, logits=tensor([[ 4.8750,  1.7880, -0.8356, -3.0027, -2.0727]],
#        grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

# The output from the model is a one-hot encoded list of scores.
# The position with the highest score represents the sentiment rating
#     e.g., [0.9, 0.2, 0.1, -0.2, -0.5]
#     is a rating of 1


SequenceClassifierOutput(loss=None, logits=tensor([[ 4.8750,  1.7880, -0.8356, -3.0027, -2.0727]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [25]:
result.logits

tensor([[ 4.8750,  1.7880, -0.8356, -3.0027, -2.0727]],
       grad_fn=<AddmmBackward0>)

In [27]:
# 用于找出给定张量（tensor）中沿指定维度的最大值的索引
int(torch.argmax(result.logits)) + 1

1

In [2]:
df = pd.read_csv('D:\Study\ORIE_24SP\MLB-Final-Project\Customer_Reviews.csv')

In [4]:
# 加载预训练的 tokenizer 和模型
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 检查是否有可用的 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def batch_sentiment_analysis(reviews, tokenizer, model, batch_size=32):
    # 将评论分成多个批次
    batches = [reviews[i:i + batch_size] for i in range(0, len(reviews), batch_size)]
    results = []

    for batch in batches:
        # 编码
        encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
        encoded_input = {k: v.to(device) for k, v in encoded_input.items()}  # 移动到 GPU
        
        # 预测
        with torch.no_grad():
            outputs = model(**encoded_input)
        
        # 获取预测的类别索引
        predictions = torch.argmax(outputs.logits, dim=-1)
        results.extend(predictions.cpu().numpy())  # 移回 CPU

    return np.array(results) + 1  # 转换为情感评分

# 使用例子
reviews = ["I love this product!", "This is the worst thing I ever bought."]
sentiments = batch_sentiment_analysis(reviews, tokenizer, model)
print(sentiments)


d:\Study\ORIE_24SP\PAML\streamlitenv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[5 1]


In [5]:
# 假设 df 是您的 DataFrame，且评论存储在 'review_text' 列中
sentiments = batch_sentiment_analysis(df['review_text'].tolist(), tokenizer, model)
df['sentiment_score'] = sentiments

In [6]:
df

,customer_id,product_id,review_text,sentiment_score
0,fddf62ef-6c1c-4ec2-8cf1-25be9cba0909,e402ec1b-e4de-43f3-b59a-f9ba9b32e1a0,Not what I expected.,2
1,96e8152a-f5be-468f-b72b-e12da2e57aa6,9267989e-c04f-4453-b608-6781177744bf,Not what I expected.,2
2,57d56731-a79b-4516-8248-6f243022e769,06edb8f1-4fd4-4298-a91d-e32fba9fef18,Could be better.,3
3,55fdde64-c608-4691-889f-d3490310e8b6,894374ff-0ee4-472c-b7b8-27c0879989a1,Not what I expected.,2
4,4f2b70d7-dc65-4c2d-bc62-11c4c7745e55,c68d0754-2b06-41f8-a60f-56006460ba5b,Worst purchase ever!,1
...,...,...,...,...
993,7c3bdab2-79e3-46a4-8e72-83441108dd3f,86601c43-5979-4ae4-b469-4b9f07cfeba8,Could be better.,3
994,b82e3f0f-ac59-461c-b4d3-6ff6ec3b8f55,37e92eba-bda1-42b3-9c4c-52dfeba774bc,Worst purchase ever!,1
995,d5e66d2c-b545-4206-a9f4-9b1c59de5eda,63b4083a-0d3a-4b9e-aa73-ac41d7d400b2,Not what I expected.,2
996,193bb323-aeec-40ab-995a-762025185d1b,55f1830f-f064-4f81-ba91-482c03af8895,Absolutely love this!,5
